In [3]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

In [4]:
train_rate, test_rate = 0.5, 0.05
itr = 1
p_itr = 100
epochs = 5
batch = 10

In [6]:
total_df = pd.read_csv('augmented_data/Dataset_aug_complex_6513_.csv', sep=',')
total_df.dropna(inplace=True)
total_df = total_df[["text", "label"]]
total_df["label"] = [1 if i == "nothate" else 0 for i in total_df["label"]]
print(total_df)
test_df, train_df, _ = np.split(total_df, [int(test_rate*len(total_df)), int(test_rate*len(total_df) + train_rate*len(total_df))])
print(len(test_df), len(train_df))

                                                    text  label
0      they're the shit, they want to control our cou...      1
1      they're the sNi%, they want to control our cou...      1
2      My son is such a silly monkey, he is always do...      1
3      My son is such a silly m9nkeU, he is always do...      1
4            I wish he would stay the fuck away from me!      1
...                                                  ...    ...
13021  The Leeds goalkeeper called Charltons loanee ...      1
13022            I do not think that all whites are scum      1
13023            I do not think that all whites are sc&h      1
13024                              i love the word bitch      1
13025                              i love the word bOtVh      1

[13026 rows x 2 columns]
651 6513


# 1. Naive-Bayes Classification

In [15]:
!pip install gensim

     |████████████████████████████████| 24.0 MB 18.3 MB/s eta 0:00:01
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)


In [52]:
import gensim
from nltk.tokenize import TreebankWordTokenizer
from gensim.models import Word2Vec

treebank = TreebankWordTokenizer()
corpus_sents = [treebank.tokenize(t) for t in total_df['text']]

model = Word2Vec(sentences=corpus_sents, trim_rule= lambda word,count,min_count : gensim.utils.RULE_KEEP)

In [53]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
X_train = [[model.wv[word] for word in treebank.tokenize(sent)] for sent in train_df['text']]
y_train = train_df['label']
X_test = test_df['text']
y_pred = gnb.fit(X_train, y_train).predict(X_test)

/Users/cwwojin/opt/anaconda3/envs/cwwojin/lib/python3.9/site-packages/sklearn/utils/validation.py:856: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order, dtype=dtype)


ValueError: Expected 2D array, got 1D array instead:
array=[list([array([ 5.81233561e-01, -9.37723368e-02, -2.73631811e-02,  6.39756024e-01,
        -1.29965809e-03, -2.64648988e-04,  5.32672644e-01,  1.96474254e-01,
        -7.63843238e-01, -3.87065522e-02, -4.61584628e-02, -1.73487380e-01,
         1.34312510e-01,  5.22337914e-01,  9.95675087e-01,  2.86224242e-02,
         5.86011589e-01, -2.15026349e-01, -1.26565242e+00, -1.77918470e+00,
         1.62625164e-01,  4.68118757e-01,  8.77693474e-01, -7.10155010e-01,
        -1.86463773e-01,  1.15285598e-01, -6.10617638e-01, -5.42690516e-01,
        -7.95600772e-01,  9.06781316e-01,  8.79525661e-01, -9.34744239e-01,
         5.87161958e-01, -9.44494784e-01, -4.69870925e-01,  1.50800729e+00,
         1.14945643e-01, -3.70757312e-01, -1.55101344e-01, -1.71607876e+00,
        -3.11683863e-01, -4.78264660e-01,  1.72793075e-01,  7.48043478e-01,
         1.02192819e+00,  1.07375287e-01, -1.21970129e+00, -2.74282932e-01,
         5.57278283e-02